In [26]:
!pip install alpaca-py
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import datetime
import numpy as np
import time


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# ✅ Step 2: Set your API credentials
API_KEY = "PKW3AW0TU6XNQ32SHI20"
API_SECRET = "lrH9p5SG9mtINYVutcNBA1RBcBi1vsOqOmsOs1SJ"

# ✅ Step 3: Initialize client
client = StockHistoricalDataClient(API_KEY, API_SECRET)

In [28]:

assets = [
    # Tech
    "AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NVDA", "ADBE", "CRM", "INTC", "ORCL",
    # Finance
    "JPM", "BAC", "GS", "WFC", "MS", "AXP", "C", "BLK", "SCHW", "USB",
    # Energy
    "XOM", "CVX", "BP", "TTE", "COP", "PSX", "SLB", "HAL", "VLO", "MPC",
    # Healthcare
    "JNJ", "PFE", "MRK", "UNH", "LLY", "BMY", "ABBV", "CVS", "CI", "AMGN",
    # Consumer/Retail
    "T", "VZ", "NFLX", "DIS", "NKE", "KO", "PEP", "MCD", "LOW", "HD"
]


In [29]:
# ✅ Step 5: Define time range
start_date = datetime.date(2015, 1, 1)
end_date = datetime.date(2023, 12, 31)

# ✅ Step 6: Download and store all data
all_data = []

for ticker in assets:
    print(f"Downloading {ticker}...")
    req = StockBarsRequest(
        symbol_or_symbols=ticker,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )
    bars = client.get_stock_bars(req).df

    if not bars.empty:
        bars['Ticker'] = ticker
        bars.reset_index(inplace=True)
        all_data.append(bars)
    time.sleep(1)  # prevent rate limiting

# ✅ Step 7: Combine and save
if all_data:
    full_df = pd.concat(all_data)
    full_df.to_csv("../data/price_data_flat.csv", index=False)
    print("✅ Data saved to ../data/price_data_flat.csv")
else:
    print("❌ No data downloaded.")






✅ Data saved to ../data/price_data_flat.csv


In [34]:
flat_df = pd.read_csv("../data/price_data_flat.csv")

# ✅ Step 6: Compute returns
returns = flat_df.pivot(index='timestamp', columns='Ticker', values='close').pct_change().dropna()
returns.to_csv("../data/returns.csv")
print("✅ Daily returns saved to ../data/returns.csv")


✅ Daily returns saved to ../data/returns.csv


In [31]:

sector_df = pd.DataFrame.from_dict(sector_map, orient='index', columns=['Sector'])
sector_df.index.name = 'Ticker'
sector_df.to_csv("../data/sector_map.csv")
print("✅ Sector map saved to ../data/sector_map.csv")

✅ Sector map saved to ../data/sector_map.csv


In [32]:
price_df = pd.read_csv("../data/price_data_flat.csv", index_col=0, parse_dates=True)
price_df.head()

C:\Users\ayush\AppData\Local\Temp\ipykernel_27364\2076018149.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  price_df = pd.read_csv("../data/price_data_flat.csv", index_col=0, parse_dates=True)


,timestamp,open,high,low,close,volume,trade_count,vwap,Ticker
symbol,,,,,,,,,
AAPL,2016-01-04 05:00:00+00:00,102.61,105.368,102.00,105.35,71935339.0,351452.0,104.007250,AAPL
AAPL,2016-01-05 05:00:00+00:00,105.75,105.850,102.41,102.71,58690536.0,321365.0,103.414304,AAPL
AAPL,2016-01-06 05:00:00+00:00,100.56,102.370,99.87,100.70,71079827.0,409164.0,100.847762,AAPL
AAPL,2016-01-07 05:00:00+00:00,98.68,100.130,96.43,96.45,85996453.0,462836.0,98.046209,AAPL
AAPL,2016-01-08 05:00:00+00:00,98.55,99.110,96.76,96.96,75066292.0,419555.0,97.916490,AAPL
